# ACCESS-NRI 2025 -- CMWG Workshop 

# Visualising ISSM Models - *Part 2*

`pyISSM` has a series of efficient visualisation tools, accessible via the `pyissm.plot` submodule. Key visualisation tools currently include:
- Mesh visualisation
- Boundary conditions
- 2D field visualisation (input and results)
- Transient results (1D timeseries)

All pyISSM visualisation capabilities leverage existing Python modules (e.g. `matplotlib` and `xarray`). Users can create complete complex and publication-ready figures using familiar syntax. By default, all `pyISSM` visualisation functions return a figure and set of axes, consistent with `matplotlib.pyplot.subplots()`. Optionally, an existing axes object can be passed to the `pyISSM` function for more refined control of plotting.

<div class="alert alert-block alert-info">
<b>AIM:</b> This notebook focuses on visualising ISSM model inputs and results.
</div>

In [ ]:
# Import required modules
import matplotlib.pyplot as plt
import matplotlib.patches
import numpy as np


# Import pyISSM
# NOTE: Here, we assume that the `pyISSM` git repository has been cloned onto your machine. If `pyISSM` has been installed using `conda`, skip the `sys` calls.
import sys
import socket
hostname = socket.gethostname()
if 'gadi' in hostname:
    sys.path.append('/home/565/lb9857/gitRepos/pyISSM/src/')
else:
    sys.path.append('/Users/lawrence.bird/pyISSM/src/')
    import matplotlib
    matplotlib.use('MacOSX')
    %matplotlib inline
import pyissm

In [ ]:
# Load an existing ISSM model onto md

if 'gadi' in hostname:
    md = pyissm.model.io.load_model('/home/565/lb9857/gitRepos/CMWG_workshop_2025/sampleModels/Greenland.HistoricTransient_200yr.nc')
else:
    md = pyissm.model.io.load_model('/Users/lawrence.bird/access-dev/sampleModels/Greenland.HistoricTransient_200yr.nc')

## 1. Visualise key model input fields

To visualise 2D model fields, we can used the `pyissm.plot.plot_model_field()` function. This can be used to visualise any static 2D field within an ISSM model (e.g. `md.geometry.surface`, `md.results.StressbalanceSolution.Vel`). It can also be used to visualise snapshots from `md.results.Transientsolution` (e.g. `md.results.TransientSolution.Vel[0]`).

Below, let's plot the ice base field and explore the capability of `pyissm.plot.plot_model_field()`.

In [ ]:
# Default 2D field plot
fig, ax = pyissm.plot.plot_model_field(md, md.geometry.base, figsize = (4, 6))

As before, we can add additional complexity to these 2D field plots. Below, let's add some more complexity:

- **Panel 1**: We add the model mesh using the `show_mesh = True` option, and pass arguments to adjust the mesh appearance by providing a dictionary to `mesh_args`. We can pass nested dictionaries to add and control the appearance of the nodes.

- **Panel 2**: We adjust the visualisation of the data to plot it on 'elements', rather than 'points' (which is the default). This displays the average value of the three points associated with each triangle.

- **Panel 3**: We add a colorbar to show the data values, and pass arguments to add a colorbar label by providing a dictionary to `cbar_args`.

In [ ]:
# Initialise a figure with 3 panels
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, sharex = True, sharey = True, figsize=(10, 5), constrained_layout=True)

# Add mesh, with optional arguments
ax1 = pyissm.plot.plot_model_field(md,
                                   md.geometry.base,
                                   ax = ax1,
                                   ylabel = 'Model Y coordinate (m)',
                                   xlabel = 'Model X coordinate (m)',
                                   show_mesh = True,
                                   mesh_args = {'color': 'blue',
                                                'linewidth': 0.2,
                                                'show_nodes': True,
                                                'node_args': {'color': 'red',
                                                              's': 1}})

# Plot data on elements
ax2 = pyissm.plot.plot_model_field(md,
                                   md.geometry.base,
                                   ax = ax2,
                                   ylabel = '',
                                   xlabel = 'Model X coordinate (m)',
                                   plot_data_on = 'elements')

# Add colorbar to show data values, with optional arguments
ax3 = pyissm.plot.plot_model_field(md,
                                   md.geometry.base,
                                   ax = ax3,
                                   ylabel = '',
                                   xlabel = 'Model X coordinate (m)',
                                   show_cbar = True,
                                   cbar_args = {'label': 'Ice base elevation (m)'})

# Adjust the X/Y ticks (only need to adjust ax1 as all subplots share X/Y axes)
ax1.set_xticks([-5*1e5, 0, 5*1e5])
ax1.set_xticklabels(['-500', '0', '500'])
ax1.set_yticks([-1*1e6, -2*1e6, -3*1e6])
ax1.set_yticklabels(['-1000', '-2000', '-3000'])

plt.show()

# 2. Visualise model results

## 2.1 - 2D fields

We can use a similar appraoch to visualise the change in a field over time. Below, let's look at thickness changes after 200 years. Below, we introduce some additional uses of `pyissm.plt.plot_model_field()`:

- We can limit the extent of data displayed by passing the `vmin` and `vmin` kwargs. Associated with this, we can extend the colorbar by including the `extend` option to the `cbar_args` dictionary.

- We can modify the colormap of the difference plot by passing the `cmap` kwarg.

In [ ]:
# Caculate the change in ice thickness over the 200 year simulation
thickness_change = md.results.TransientSolution.Thickness[-1] - md.geometry.thickness

# Initialise a figure with 3 panels to visualise the change in ice thickness
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, sharex = True, sharey = True, figsize=(10, 6), constrained_layout=True)

# Plot initial ice thickness
ax1 = pyissm.plot.plot_model_field(md,
                                   md.geometry.thickness,
                                   ax = ax1,
                                   vmin = 0,
                                   vmax = 3000,
                                   ylabel = 'Model Y coordinate (m)',
                                   xlabel = 'Model X coordinate (m)',
                                   show_cbar = True,
                                   cbar_args = {'label': 'Ice thickness (m)',
                                                'orientation': 'horizontal',
                                                'extend': 'max'})

# Plot final ice thickness
ax2 = pyissm.plot.plot_model_field(md,
                                   md.results.TransientSolution.Thickness[-1],
                                   ax = ax2,
                                   vmin = 0,
                                   vmax = 3000,
                                   ylabel = '',
                                   xlabel = 'Model X coordinate (m)',
                                   show_cbar = True,
                                   cbar_args = {'label': 'Ice thickness (m)',
                                                'orientation': 'horizontal',
                                                'extend': 'max'})

# Plot change in ice thickness
ax3 = pyissm.plot.plot_model_field(md,
                                   thickness_change,
                                   ax = ax3,
                                   cmap = 'RdBu',
                                   vmin = -200,
                                   vmax = 200,
                                   ylabel = '',
                                   xlabel = 'Model X coordinate (m)',
                                   show_cbar = True,
                                   cbar_args = {'label': 'Thickness change (m)',
                                                'extend': 'both'})

# Adjust the X/Y ticks (only need to adjust ax1 as all subplots share X/Y axes)
ax1.set_xticks([-5*1e5, 0, 5*1e5])
ax1.set_xticklabels(['-500', '0', '500'])
ax1.set_yticks([-1*1e6, -2*1e6, -3*1e6])
ax1.set_yticklabels(['-1000', '-2000', '-3000'])

# Add titles to each subplot
ax1.set_title('Initial ice thickness', fontsize = 10)
ax2.set_title('Final ice thickness', fontsize = 10)
ax3.set_title('Thickness change (200 years)', fontsize = 10)

plt.show()

In [ ]:
# Caculate the change in ice thickness over the 200 year simulation
thickness_change = md.results.TransientSolution.Thickness[-1] - md.geometry.thickness

# Initialise a figure with 3 panels to visualise the change in ice thickness
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, sharex = True, sharey = True, figsize=(10, 6), constrained_layout=True)

# Plot initial ice thickness
ax1 = pyissm.plot.plot_model_field(md,
                                   md.geometry.thickness,
                                   ax = ax1,
                                   vmin = 0,
                                   vmax = 3000,
                                   ylabel = 'Model Y coordinate (m)',
                                   xlabel = 'Model X coordinate (m)',
                                   show_cbar = True,
                                   cbar_args = {'label': 'Ice thickness (m)',
                                                'orientation': 'horizontal',
                                                'extend': 'max'})

# Plot final ice thickness
ax2 = pyissm.plot.plot_model_field(md,
                                   md.results.TransientSolution.Thickness[-1],
                                   ax = ax2,
                                   vmin = 0,
                                   vmax = 3000,
                                   ylabel = '',
                                   xlabel = 'Model X coordinate (m)',
                                   show_cbar = True,
                                   cbar_args = {'label': 'Ice thickness (m)',
                                                'orientation': 'horizontal',
                                                'extend': 'max'})

# Plot change in ice thickness
ax3 = pyissm.plot.plot_model_field(md,
                                   thickness_change,
                                   ax = ax3,
                                   cmap = 'RdBu',
                                   vmin = -200,
                                   vmax = 200,
                                   ylabel = '',
                                   xlabel = 'Model X coordinate (m)',
                                   show_cbar = True,
                                   cbar_args = {'label': 'Thickness change (m)',
                                                'extend': 'both'})

# Adjust the X/Y ticks (only need to adjust ax1 as all subplots share X/Y axes)
ax1.set_xticks([-5*1e5, 0, 5*1e5])
ax1.set_xticklabels(['-500', '0', '500'])
ax1.set_yticks([-1*1e6, -2*1e6, -3*1e6])
ax1.set_yticklabels(['-1000', '-2000', '-3000'])

# Add titles to each subplot
ax1.set_title('Initial ice thickness', fontsize = 10)
ax2.set_title('Final ice thickness', fontsize = 10)
ax3.set_title('Thickness change (200 years)', fontsize = 10)

plt.show()

# 2.2 - 1D timeseries

In addition to 2D model fields, `pyISSM` also provides efficient tools for plotting timeseries output provided by ISSM. Here, we introduce the `pyissm.plot.plot_model_ts()` function.

In its simplest form, `pyissm.plot.plot_model_ts()` generates a figure with individual subplots for all 1D timeseries included in `md.results.TransientSolution`. We demonstrate this below.

In [ ]:
# Default 1D timeseries plot
# This will generate a figure with individual subplots for all 1D timeseries included in `md.results.TransientSolution`

fig, ax = pyissm.plot.plot_model_ts(md)

Users can plot a subset of model results by providing a `data_list` (and associated optional `variable_list` and `unit_list`). This appraoch allows users to specify data series with non-standard ISSM units. We demonstrate this below.

In [ ]:
# Users can plot a subset of model results by providing a `data_list` (and associated optional `variable_list` and `unit_list`).
# This appraoch allows users to specify data series with non-standard ISSM units.

# Convert Ice Volume from m^3 to km^3
select_data = [md.results.TransientSolution.IceVolume / 1e9, md.results.TransientSolution.TotalSmb]

# Pass associated variable and unit names
variable_names = ['Ice volume', 'Total SMB']
unit_names = ['km$^3$', 'Gt yr$^{-1}$']

fig, ax = pyissm.plot.plot_model_ts(md,
                                    data_list = select_data,
                                    variable_list = variable_names,
                                    unit_list = unit_names,
                                    color = 'green',
                                    linewidth = 4,
                                    linestyle = '--')